# QuickGuide for Text analysis
[프로젝트 페이지: Research-on-the-TV-market](https://github.com/xikest/research-market-tv)

## Env setting: Install Colab selenium & crome driver

In [1]:
!pip install -U pandas openpyxl tqdm
!pip install -U requests selenium beautifulsoup4 
!pip install -U wordcloud nltk 
!pip install -U scikit-learn openai 
!pip install -U matplotlib seaborn
!pip install -U mkt-retv

## Env 세팅

In [2]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from pathlib import Path
import pandas as pd
from google.colab import files
from market_research import SentiGPT
from market_research import TextAnalysis

### 분석 폴더
- `input_text`에 파일을 넣어주세요.

In [6]:
intput_folder = Path("input_text")  # 폴더 이름을 지정

# 폴더가 존재하지 않으면 폴더 생성
if not intput_folder.exists():
    intput_folder.mkdir(parents=True)

output_folder = Path('results_text')  
if not output_folder.exists():
  output_folder.mkdir(parents=True, exist_ok=True)

## 준비

- text 분석할 `column`은 `sentences`입니다.

In [11]:
# 분석할 엑셀 파일이 있는 폴더 경로
file_list = intput_folder.glob('*')
excel_files = [file for file in file_list if file.suffix in {'.xlsx', '.xls'}]
for excel_file in excel_files:
    print(excel_file.name)

## Text 분석

In [12]:
tas = TextAnalysis()

In [32]:
for excel_file in excel_files:
    file_name=excel_file.name.split(".")[0]
    output_file_path = output_folder/file_name
    
    print(f"\n■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■[{file_name}]■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■\n")
    # 데이터 셋
    excel_file_path = excel_file
    df_uploaded = pd.read_excel(excel_file_path)
    df_sentences = df_uploaded

    #text 읽기
    comments = [sentence for sentence in df_uploaded["sentences"]]
    tas.get_comments(comments)
    tas.save_df_freq_as_excel(output_file_path, file_name)

    #그래프 만들기
    tas.plot_freq()
    tas.plot_wordcloud()

- colab에서 파일 받기

In [ ]:
!zip -r /content/results_text.zip /content/results_text/
files.download('/content/results_text.zip')

## sentgpt: Fine tune 필요

### GPT setting
- [GPT model info](https://platform.openai.com/docs/models/continuous-model-upgrades)

In [14]:
API_KEY = 
gpt_model="gpt-3.5-turbo-1106"

In [8]:
stm = SentiGPT(api_key=API_KEY, gpt_model=gpt_model)

In [10]:
for excel_file in excel_files:
    # 데이터 셋
    excel_file_path = excel_file
    df_uploaded = pd.read_excel(excel_file_path)
    df_sentences = df_uploaded
    
    # GPT 분석
    sentences_list = [sentence for sentence in df_sentences["sentences"]]  # 리스트로 변환
    keywords_list=["brightness", "color", "contrast", "reflection", "viewing angle"]
    analyzed_results_df = stm.analyze_sentences(sentences_list, keywords_list)
    
    # 분석 파일 저장
    file_name=excel_file_path.name.split(".")[0]
    output_file_path = output_folder/file_name
 
    if not output_file_path.exists():
        output_file_path.mkdir(parents=True, exist_ok=True)
    analyzed_results_df.to_csv(output_file_path / f"analyzed_results_{file_name}.csv", index=False, encoding='utf-8')
    
    #그래프 만들기
    # df_analyzed_results = analyzed_results_df - 5  #그래프 범위 조정
    print(f"{file_name}, Plot")
    stm.plot_hist_all(output_folder=output_file_path, file_name=file_name)
    stm.plot_hist_each(output_folder=output_file_path, file_name=file_name)

---